In [1]:
# from https://github.com/JerryBubble/scveloSkel/blob/main/veloSkeleton_explore.ipynb
import scvelo as scv
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')  # for beautified visualization

In [2]:
# standard preprocessing steps from scVelo
adata2000 = scv.datasets.pancreas()
scv.pp.filter_genes(adata2000, min_shared_counts=20) 
scv.pp.normalize_per_cell(adata2000)
scv.pp.filter_genes_dispersion(adata2000, n_top_genes=2000) # only leave the top 2000 genes for this run for simplicity
scv.pp.log1p(adata2000)
scv.pp.moments(adata2000, n_pcs=30, n_neighbors=30)

100%|██████████| 50.0M/50.0M [00:08<00:00, 6.43MB/s]
/Users/kevinlin/miniconda3/envs/veloUncertainty/lib/python3.9/site-packages/anndata/_core/anndata.py:1105: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/Users/kevinlin/miniconda3/envs/veloUncertainty/lib/python3.9/site-packages/anndata/_core/anndata.py:1105: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/Users/kevinlin/miniconda3/envs/veloUncertainty/lib/python3.9/site-packages/anndata/_core/anndata.py:1105: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Filtered out 20801 genes that are detected 20 counts (shared).
Normalized count data: X, spliced, unspliced.


/Users/kevinlin/miniconda3/envs/veloUncertainty/lib/python3.9/site-packages/anndata/_core/anndata.py:1105: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Extracted 2000 highly variable genes.
computing neighbors
    finished (0:00:06) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [4]:
# Using dynamical model (see paper) to calculate gene velocity
scv.tl.recover_dynamics(adata2000, n_jobs=8)

recovering dynamics (using 1/12 cores)


/Users/kevinlin/miniconda3/envs/veloUncertainty/lib/python3.9/site-packages/anndata/_core/anndata.py:1105: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/Users/kevinlin/miniconda3/envs/veloUncertainty/lib/python3.9/site-packages/anndata/_core/anndata.py:1105: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/Users/kevinlin/miniconda3/envs/veloUncertainty/lib/python3.9/site-packages/anndata/_core/anndata.py:1105: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/Users/kevinlin/miniconda3/envs/veloUncertainty/lib/python3.9/site-packages/anndata/_core/anndata.py:1105: De

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.

In [ ]:
scv.tl.velocity(adata2000, mode='dynamical')

In [ ]:
# computing velocity graph
# this gives the sparse matrix with cosine correlations (adata.uns)
scv.tl.velocity_graph(adata2000, n_jobs=8)

In [ ]:
# to look at the AnnData object with the processed results
adata2000

# Starting the skeleton construction

In [ ]:
import skeleton_methods.skeleton as skel
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance_matrix
from scipy.stats import gaussian_kde
import collections
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import adjusted_rand_score
from scipy.spatial import distance_matrix
import igraph
from collections import defaultdict
from numpy.linalg import norm

import random
 
random.seed(123)

In [ ]:
# define the knot construction function
def constructKnots(X, centers = None, labels = None, k = None, rep = 100):
    """
      construct knots using overfitting kMeans
  
      Parameters
      ----------
      X : np.array
          the data ndarray
      centers: np.array of the knots, can be provided
      labels: np.array of the knot label that each data point belongs to
      k: the number of knots
      rep: times of random initialization for kMeans
  
      Returns
      -------
      centers: ndarray of the constructed knots
      cluster: array of knot labels to which the data points belong to
      nknots: number of knots
      withinss: within cluster sum of squares for each Voronoi cell corresponding to the knots
    """
    n, d = X.shape
    #construct knots
    if centers is None and labels is None:
        # Overfitting k-means
        #setting the number of knots k
        if k is None:
            k = round(np.sqrt(n))
    
        km = KMeans(n_clusters = k, n_init = rep)
        km.fit(X)
        centers = km.cluster_centers_
        labels = km.labels_
    
    elif labels is None:#centers provided but not labels
        nbrs = NearestNeighbors(n_neighbors=1).fit(centers)
        labels = nbrs.kneighbors(X, return_distance=False)
        labels = np.array([sublist[0] for sublist in labels])
        k = len(centers)
          
    elif centers is None:#labels provided but not centers
        elements_count = collections.Counter(labels)
        k = len(elements_count.items())
        centers = np.array([[0.0 for col in range(d)] for row in range(k)])
        for key, value in elements_count.items():
            centers[key,] = np.mean(X[labels == key,], axis=0)
      
    else:
        k = len(centers)
        
      
    withinss = np.array([0.0]*k)
    for i in range(k):
        withinss[i] = np.sum(np.square(X[labels == i,:]-centers[i,:]))
      
    return {"centers":centers, "cluster":labels, "nknots":k, "withinss": withinss}

In [ ]:
# construct knots
conKnots = constructKnots(X, k = 100)
skeleton = conKnots
edgeWeights = skel.skelWeights(X, conKnots, wedge = ['voronoi'])
skeleton.update(edgeWeights)
X_nn = skeleton["nn"]
knots = skeleton["centers"]
kkdists = distance_matrix(knots,knots)

In [ ]:
# Compute the cluster sizes
knotSizes = np.zeros(100)
for i in range(100):
    knotSizes[i]= sum(X_nn[:,0] == i)
sum(knotSizes<5)

In [ ]:
#filter out small clusters and reconstruct
knots = np.array(knots[knotSizes>=5,:])
conKnots = constructKnots(X, centers= knots)
skeleton = conKnots
edgeWeights = skel.skelWeights(X, conKnots, wedge = ['voronoi'])
skeleton.update(edgeWeights)

In [ ]:
# recompute cluster sizes
X_nn = skeleton["nn"]
knots = skeleton["centers"]
kkdists = distance_matrix(knots,knots)
knotSizes = np.zeros(len(knots))
for i in range(len(knots)):
    knotSizes[i]= sum(X_nn[:,0] == i)

In [ ]:
# compute the voronoi similarity between knots, which is counting the data points with the pair of knots as the two closest knots.
# These weights can be used for approximate Delaunnay triangulation
def VoronSimilarity(knots, X_nn, kkdists):
    m = knots.shape[0]
    voron_weights = np.array([[0.0 for col in range(m)] for row in range(m)])
    for i in range(m-1): #loop through knots pairs
        # center1 = knots[i]
        wi1 = np.where(X_nn[:,0] == i)[0]
        wi2 = np.where(X_nn[:,1] == i)[0]
        for j in range(i+1,m):
            # center2 = knots[j]
            wj1 = np.where(X_nn[:,0] == j)[0]
            wj2 = np.where(X_nn[:,1] == j)[0]
            #data point indices within 2nn neighborhood of knots i, j
            nn2ij = np.union1d(np.intersect1d(wi1, wj2), np.intersect1d(wi2, wj1))
        
            if len(nn2ij) < 1 :#not in Delaunnay Triangulation
                voron_weights[i,j] = 0.0
                continue
        
            # Euclidean distance between two centers
            d12 = kkdists[i,j]

            #compute the Voronoi density 
            voron_weights[i,j] = len(nn2ij)/d12
    return voron_weights

voron_weights = VoronSimilarity(knots, X_nn, kkdists)
skeleton.update( {"voron_weights": voron_weights + np.transpose((voron_weights))})

In [ ]:
# To get major cell types within each voronoi cell
from collections import defaultdict
X_nn = skeleton["nn"]
knots = skeleton["centers"]

knotClusbyCell = defaultdict(lambda: "Not Present")
for i in range(len(knots)):
    df = pd.Series(adata2000.obs['clusters'][X_nn[:,0] == i])
    knotClusbyCell[i] = df.value_counts()

In [ ]:
# print to examine cell types within each region
for i in range(len(knots)):
    print(i)
    print(knotClusbyCell[i])

In [ ]:
# knots to further split and refine. Picked the cells where the second major cell type 
# has counts more than half of the most common cell type and larger than 10 in counts.
# the refine indices are manually picked for now
# but can be automated with the rules 
refineId = [17,24,35,38,55,78,81,88,99]
for id in refineId:
    knots[id,:] = np.mean( X[np.logical_and( X_nn[:,0] == id , adata2000.obs['clusters'] == knotClusbyCell[id].index[0]),:], axis= 0)
    knots = np.vstack((knots, 
                       np.mean( X[np.logical_and( X_nn[:,0] == id , adata2000.obs['clusters'] != knotClusbyCell[id].index[0]),:], axis= 0)))

In [ ]:
# recalculate skeleton quantities based on refined knots
conKnots = constructKnots(X, centers= np.array(knots))
skeleton_r = conKnots
edgeWeights = skel.skelWeights(X, conKnots, wedge = ['voronoi'])
skeleton_r.update(edgeWeights)

In [ ]:
# examine the refined skeleton
X_nn = skeleton_r["nn"]
knots = skeleton_r["centers"]
kkdists = distance_matrix(knots,knots)
knotSizes = np.zeros(len(knots))
for i in range(len(knots)):
    knotSizes[i]= sum(X_nn[:,0] == i)
min(knotSizes)

In [ ]:
# calculate the voronoi weights with refined knots
voron_weights_r = VoronSimilarity(knots, X_nn, kkdists)
skeleton_r.update( {"voron_weights": voron_weights_r + np.transpose((voron_weights_r))})

In [ ]:
# Use closest observation X for each knot for UMAP plotting. Save time for UMAP calculation
nbrs = NearestNeighbors(n_neighbors=1).fit(X)
nnx = nbrs.kneighbors(skeleton_r["centers"], return_distance=False) 

In [ ]:
# make knots larger sizes for plotting
sizes = np.array([1]*X.shape[0])
sizes[nnx[:,0]] = 10
colors = ['red' if i in nnx[:,0] else 'blue' for i in range(X.shape[0])]
fig, ax = plt.subplots()
ax.scatter(adata2000.obsm["X_umap"][:,0],adata2000.obsm["X_umap"][:,1], s=sizes ,c=colors)
for i in range(len(knots)):
    ax.annotate(str(i), (adata2000.obsm["X_umap"][:,0][nnx[i,0]], adata2000.obsm["X_umap"][:,1][nnx[i,0]]))
plt.show()